<a href="https://colab.research.google.com/github/dhdbsrlw/MO4E-devkor/blob/main/pykrx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 14.7 MB/s eta 0:00:00


In [2]:
from pykrx import stock

# tickers = stock.get_market_ticker_list(조회일자(YYYYmmdd) [,market=조회할 시장(KOSPI, KOSDAQ, ALL])
# name = get_market_ticker_name(심볼 [,market=조회할 시장(KOSPI, KOSDAQ, ALL])
# ohlcv = stock.stock.get_market_ohlcv(조회일자(YYYYmmdd) [,앞에 시작일을 넣었다면 종료일] [,심볼] )

In [3]:
# 예시
tickers = stock.get_market_ticker_list("20230925", market="KOSPI") # 코스피 종목 리스트
name = stock.get_market_ticker_name("005930") # 기업 이름
ohlcv1 = stock.get_market_ohlcv("20230925", market="KOSPI") # 특정일 코스피 종목들의 OHLCV
ohlcv2 = stock.get_market_ohlcv("20230901", "20230925", "005930") # 특정기간 특정 종목의 OHLCV

In [17]:
# 예시출력 1
tickers[:10]

['095570',
 '006840',
 '027410',
 '282330',
 '138930',
 '001460',
 '001465',
 '001040',
 '079160',
 '00104K']

In [9]:
# 예시출력 (기업이름 for 종목번호(심볼) 005930)
name[:10]

'삼성전자'

In [7]:
# 예시출력 3 (여러 종목에 대해)
ohlcv1[:10]

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
095570,4215,4300,4145,4155,141533,593860440,-1.420000
006840,18490,18490,18140,18490,3511,63911920,0.870000
027410,3555,3575,3505,3520,65612,232288350,-1.540000
282330,143900,145500,142400,144200,53239,7655457900,0.840000
138930,7080,7080,6960,7020,603854,4242108340,-0.280000
001460,400000,403500,396000,400000,29,11614000,0.000000
001465,140100,140100,140000,140000,8,1120100,-0.070000
001040,88500,93000,88500,91100,205209,18767536900,2.360000
079160,5750,5910,5660,5680,7863752,45415610100,-25.459999


In [10]:
# 예시출력 4 (한 종목에 대해 날짜별)
ohlcv2[:10]

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2023-09-01,66800,71000,66700,71000,29738235,6.128550
2023-09-04,72900,72900,70700,71200,26286495,0.281690
2023-09-05,70900,71500,70200,70700,12330239,-0.702247
2023-09-06,70700,70800,69700,70000,11414620,-0.990099
2023-09-07,70000,70600,69600,70400,13741241,0.571429
2023-09-08,70200,70300,69600,70300,10688118,-0.142045
2023-09-11,70400,70800,70000,70800,11785462,0.711238
2023-09-12,70800,71000,70400,70500,11688599,-0.423729
2023-09-13,71100,71600,70300,70900,15955797,0.567376


# Price Prediction
Reference: https://www.kaggle.com/code/raoulma/ny-stock-price-prediction-rnn-lstm-gru

1. Setting

In [11]:
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf

In [12]:
# split data in 80%/10%/10% (train/validation/test)
valid_set_size_percentage = 10
test_set_size_percentage = 10

2. Analyze Data

In [13]:
# 2023년 9월 25일 주가 데이터 - 951개의 샘플
df = stock.get_market_ohlcv("20230925", market="KOSPI")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 951 entries, 095570 to 003280
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시가      951 non-null    int32  
 1   고가      951 non-null    int32  
 2   저가      951 non-null    int32  
 3   종가      951 non-null    int32  
 4   거래량     951 non-null    int32  
 5   거래대금    951 non-null    int64  
 6   등락률     951 non-null    float32
dtypes: float32(1), int32(5), int64(1)
memory usage: 37.1+ KB


,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
095570,4215,4300,4145,4155,141533,593860440,-1.42
006840,18490,18490,18140,18490,3511,63911920,0.87
027410,3555,3575,3505,3520,65612,232288350,-1.54
282330,143900,145500,142400,144200,53239,7655457900,0.84
138930,7080,7080,6960,7020,603854,4242108340,-0.28


In [20]:
# Ticker 를 별개의 Column 으로 df 에 추가하여 Import

# Initialize
df_concat = []

for ticker in tickers:
  df['종목'] = ticker

df.head()

,시가,고가,저가,종가,거래량,거래대금,등락률,종목
티커,,,,,,,,
095570,4215,4300,4145,4155,141533,593860440,-1.42,003280
006840,18490,18490,18140,18490,3511,63911920,0.87,003280
027410,3555,3575,3505,3520,65612,232288350,-1.54,003280
282330,143900,145500,142400,144200,53239,7655457900,0.84,003280
138930,7080,7080,6960,7020,603854,4242108340,-0.28,003280


In [14]:
df.tail()

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
079980,5640,5790,5560,5650,222523,1258784360,-0.70
005010,5280,5500,5280,5410,875864,4742720670,2.27
000540,3195,3195,3020,3090,61794,192343515,-3.29
000545,5110,5110,5020,5020,1525,7746000,-3.09
003280,1707,1720,1643,1643,429667,714980126,-3.75


In [15]:
# EDA

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(df[df.티커 == 005930].시가, color='red', label='open')
plt.plot(df[df.티커 == 005930].고가, color='black', label='high')
plt.plot(df[df.티커 == 005930].저가, color='blue', label='low')
plt.plot(df[df.티커 == 005930].종가, color='green', label='close')
plt.title('stock price')
plt.xlabel('time [days]')
plt.ylabel('price')
plt.legend(loc='best')
plt.show()

SyntaxError: ignored